In [1]:
#!/usr/bin/env python3
import os, platform, subprocess, sys, requests, time

In [2]:
"""
Finding and importing provided libraries regardless of system and platform type
"""
## find and import libraries relative to this file's location (regardless of script or notebook)
## platfrom specific naming already handled within the lib directory's __init__ file
if '__file__' not in globals():
    sys.path.append(os.path.dirname(os.path.abspath('').split('transactions')[0]))
else:
    sys.path.append(os.path.dirname(__file__).split('transactions')[0])

from lib.encoder  import encode_tx, encode_script
from lib.helper   import decode_address
from lib.hash     import hash160, hash256, sha256
from lib.rpc      import RpcSocket
from lib.rpcauth  import rpcauth
from lib.sign     import sign_tx

In [3]:
'''
Step 0.0: setup our RPC socket like we did in the previous example
        If you have not done the previous example, now would be the time.
        Please copy in the variables you used (located conveniently at the end of the previous example)
'''

nodeIP = "127.0.0.1"
nodePort = "18444"
username = "user"
password = "password"
wallet = "test2"

rpc = RpcSocket({ 'wallet': wallet,
                  'username': username,
                  'password': password,
                  'url' : nodeIP,
                  'port': nodePort})
assert rpc.check()

In [4]:
'''
Step 0.1: make sure you have cash. If you are broke, see previous example on how to get funds
'''
if float(rpc.call("getbalance")) == 0:
    raise Exception("get funds OR check if funds are still in a pending (ie. unconfirmed) state")
else:
    print("You have a balance of:", rpc.call("getbalance"), "BTC")

You have a balance of: 0.02980655 BTC


In [5]:
'''
First, we will lookup an existing utxo, and use that to fund our transaction.
'''
utxo = rpc.get_utxo(0)
print(utxo)

{'address': 'tb1qwjkj7jev5y8zegqz34t52ulwm5wsvhtleukcfa', 'priv_key': 'c97b144b8e8e4664bf684ec9ababa745b7f754667b7ac2c0acfd52e22850590d', 'pub_key': '02da7831ee560737900635f4ffc97f2f0ac452ffa4f9525e6c6368f59938a7fdbd', 'pubkey_hash': '74ad2f4b2ca10e2ca0028d574573eedd1d065d7f', 'redeem_script': '1976a91474ad2f4b2ca10e2ca0028d574573eedd1d065d7f88ac', 'txid': '07e555a4400096be80ac7247d26d394ee6a7b733bcd79aab4d90de497d7d4bfc', 'vout': 0, 'value': 2980655}


In [6]:
## We will also grab a new receiving address,
## and lock the funds to this address.
recv = rpc.get_recv()
print(recv)

{'address': 'tb1q90ltvsgaeyxanzz2m9fdeerg9s2mjn709s7ukg', 'priv_key': '361dbd80ad9449a781eb74f334d8ca963c6461814849b0bc83b9811ceb91123f', 'pub_key': '03bf33f1d559708db5c824d8139ca25f3c55d99b6bd20c844d46596226fdf545d9', 'pubkey_hash': '2bfeb6411dc90dd9884ad952dce4682c15b94fcf', 'redeem_script': '1976a9142bfeb6411dc90dd9884ad952dce4682c15b94fcf88ac'}


In [7]:
## Replace this default preimage with your own secret.
secret_preimage = 'weareallsatoshi'

## Convert the secret to bytes, then hash using hash160 function.
secret_bytes = secret_preimage.encode('utf8').hex()
secret_hash  = hash160(secret_bytes).hex()

In [8]:
## This is where we specify the version number for the program interpreter. 
## We'll be using version 0.
script_version = 0

## Here is the locking script that we will be using. We are going to
## require the redeemer to reveral the secret, along with their public
## key for the receipt address, and matching signature.
script_words = [
    'OP_HASH160', secret_hash, 'OP_EQUALVERIFY', 
    'OP_DUP', 'OP_HASH160', hash160(recv['pub_key']).hex(), 'OP_EQUALVERIFY', 
    'OP_CHECKSIG',
]

In [9]:
## This is the hex-encoded format of the script. We will present this when 
## we unlock and spend the output. It should match the pre-image used for 
## making the script hash.
redeem_script = encode_script(script_words, prepend_len=False).hex()

## We hash the script using sha256, then provide a version number
## along with the hash. This will lock the transaction output to 
## accept only the program script which matches the hash.
script_hash = sha256(redeem_script).hex()

## Calculate the value of the transaction output, minus fees.
locking_tx_value = utxo['value'] - 1000

In [10]:
## The initial locking transaction. This spends the utxo from our funding 
## transaction, and moves the funds to the utxo for our witness program.
locking_tx = {
    'version': 1,
    'vin': [{
        'txid': utxo['txid'],
        'vout': utxo['vout'],
        'script_sig': [],
        'sequence': 0xFFFFFFFF
    }],
    'vout': [{
        'value': locking_tx_value,
        'script_pubkey': [ script_version, script_hash ]
    }],
    'locktime': 0
}

In [11]:
## Encode the transaction into raw hex,
## and calculate the transaction ID
locking_hex  = encode_tx(locking_tx)
locking_txid = hash256(bytes.fromhex(locking_hex))[::-1].hex()

In [12]:
## Sign the transaction using our key-pair from the utxo.
locking_sig = sign_tx(
  locking_tx,               # The transaction.
  0,                        # The input being signed.
  utxo['value'],            # The value of the utxo being spent.
  utxo['pubkey_hash'],
  utxo['priv_key']          # The private key to the utxo pubkey hash.
)

In [13]:
## Add the signature and public key to the transaction.
locking_tx['vin'][0]['witness'] = [ locking_sig, utxo['pub_key'] ]

In [14]:
print(f'''
# Pay-to-Witness-Script-Hash Example

Locking Txid:
{locking_txid}

Redeem Script:
{redeem_script}

Locking Tx:
{encode_tx(locking_tx)}
''')


# Pay-to-Witness-Script-Hash Example

Locking Txid:
aceb17b4f47885f8c201206a0feb47f8431c9f31d8207e20fc5010c0f67a4b0b

Redeem Script:
a91478d574f6f76809749cb351af7e20ae52082e2e378876a9142bfeb6411dc90dd9884ad952dce4682c15b94fcf88ac

Locking Tx:
01000000000101fc4b7d7d49de904dab9ad7bc33b7a7e64e396dd24772ac80be960040a455e5070000000000ffffffff0147772d00000000002200200165dc58dd14e441cf3079e135797c4ea1192953b4cd57a9d0314e7f2cb8f81d02473044022011df3095d857e54fa61f823aea88e767dbf90637a84b949f1b55e3351388fa8f0220259fe9e49a3db7f24accdb6406480154973853f5611e0f9e537876a205fab864012102da7831ee560737900635f4ffc97f2f0ac452ffa4f9525e6c6368f59938a7fdbd00000000



In [15]:
## send it off :D
locking_tx_id = rpc.transact(encode_tx(locking_tx))
print(locking_tx_id)

aceb17b4f47885f8c201206a0feb47f8431c9f31d8207e20fc5010c0f67a4b0b


In [16]:
## Bech32 addresses will decode into a script version and pubkey hash.
script_version, pubkey_hash = decode_address(recv['address'])

In [17]:
## This transaction will redeem the previous utxo by providing the secret 
## pre-image, plus the public key and signature, plus the witness program. 
## Once the transaction is confirmed, your wallet software should recognize 
## this utxo as spendable.
redeem_tx = {
    'version': 1,
    'vin': [{
        'txid': locking_txid,
        'vout': 0,
        'script_sig': [],
        'sequence': 0xFFFFFFFF
    }],
    'vout': [{
        'value': locking_tx_value - 1000,
        'script_pubkey': [ script_version, pubkey_hash ]
    }],
    'locktime':0
}

redeem_sig = sign_tx(
  redeem_tx,
  0,
  locking_tx_value,
  redeem_script,
  recv['priv_key']
)

In [18]:
redeem_tx['vin'][0]['witness'] = [ redeem_sig, recv['pub_key'], secret_bytes, redeem_script ]

print(f'Unlocking Tx:\n{encode_tx(redeem_tx)}')

Unlocking Tx:
010000000001010b4b7af6c01050fc207e20d8319f1c43f847eb0f6a2001c2f88578f4b417ebac0000000000ffffffff015f732d00000000001600142bfeb6411dc90dd9884ad952dce4682c15b94fcf04483045022100eddf6614990330259823b3443ad806d83158869dd6d876e82835eb1f2aad711a022004978dc709549d19dea33a4455ee21b02f1b60f1b6139a83e35eb56aed45a4df012103bf33f1d559708db5c824d8139ca25f3c55d99b6bd20c844d46596226fdf545d90f7765617265616c6c7361746f73686930a91478d574f6f76809749cb351af7e20ae52082e2e378876a9142bfeb6411dc90dd9884ad952dce4682c15b94fcf88ac00000000


In [19]:
## redeem it
redeem_tx_id = rpc.transact(encode_tx(redeem_tx))
print(redeem_tx_id)

41cd82ced4e980551315aca56501fc02e98568d4b6b7d7ae2de32ece9cea280b


Bonus: Fun with mempool

https://mempool.space/ is a site that you can look up the state of various popular blockchains


In [20]:
'''
The original utxo that funded the transaction can be found here
'''
url = "https://mempool.space/testnet/tx/" + utxo['txid']
print("https://mempool.space/testnet/tx/" + utxo['txid'])

https://mempool.space/testnet/tx/07e555a4400096be80ac7247d26d394ee6a7b733bcd79aab4d90de497d7d4bfc


In [21]:
'''
The original utxo that funded the transaction can be found here
We have obtained several fields about this utxo.
First, let's find this transaction on testnet mem pool, using its REST api. And print some stats.
'''
print("https://mempool.space/testnet/tx/" + utxo['txid'])
api_url = "https://mempool.space/testnet/api/tx/" + utxo['txid']
response = requests.get(api_url)
response.json()

https://mempool.space/testnet/tx/07e555a4400096be80ac7247d26d394ee6a7b733bcd79aab4d90de497d7d4bfc


{'txid': '07e555a4400096be80ac7247d26d394ee6a7b733bcd79aab4d90de497d7d4bfc',
 'version': 1,
 'locktime': 0,
 'vin': [{'txid': '6f0899c18c5ae15120c28e80fe28a5d4353e1e077a4d86862e139ae80e5143e1',
   'vout': 0,
   'prevout': {'scriptpubkey': '00200e8758dadd874a46942b5cdf921a3948434ba4933bbe083950a76546ccb606f1',
    'scriptpubkey_asm': 'OP_0 OP_PUSHBYTES_32 0e8758dadd874a46942b5cdf921a3948434ba4933bbe083950a76546ccb606f1',
    'scriptpubkey_type': 'v0_p2wsh',
    'scriptpubkey_address': 'tb1qp6r43kkasa9yd9pttn0eyx3efpp5hfyn8wlqsw2s5aj5dn9kqmcscn29mv',
    'value': 2981655},
   'scriptsig': '',
   'scriptsig_asm': '',
   'witness': ['3045022100d4e6a38d8eb1409d32153a8ba11fd2f8de74e600191c06bc48b352eae998bda30220764dfde3b07ca5d46966f56fcf9f06f9a4086783e3e6834a7ec2dac2eb0fb35101',
    '02da7831ee560737900635f4ffc97f2f0ac452ffa4f9525e6c6368f59938a7fdbd',
    '7765617265616c6c7361746f736869',
    'a91478d574f6f76809749cb351af7e20ae52082e2e378876a91474ad2f4b2ca10e2ca0028d574573eedd1d065d7f88ac']

In [22]:
'''
The mem pool REST api can also tell us the status of this transaction
'''
api_url = "https://mempool.space/testnet/api/tx/" + utxo['txid'] + "/status"
response = requests.get(api_url)
response.json()

{'confirmed': True,
 'block_height': 2410549,
 'block_hash': '000000000fffd92e16bcfc085724bf90bb1754696da44b929dd3bff584db1704',
 'block_time': 1670619190}

In [23]:
'''
Next, let's find the address on testnet mem pool.
'''
api_url = "https://mempool.space/testnet/api/address/" + utxo['address']
response = requests.get(api_url)
response.json()

{'address': 'tb1qwjkj7jev5y8zegqz34t52ulwm5wsvhtleukcfa',
 'chain_stats': {'funded_txo_count': 1,
  'funded_txo_sum': 2980655,
  'spent_txo_count': 0,
  'spent_txo_sum': 0,
  'tx_count': 1},
 'mempool_stats': {'funded_txo_count': 0,
  'funded_txo_sum': 0,
  'spent_txo_count': 0,
  'spent_txo_sum': 0,
  'tx_count': 0}}

In [24]:
'''
We can also get the transaction history for this address
'''
api_url = "https://mempool.space/testnet/api/address/" + utxo['address'] + '/txs'
response = requests.get(api_url)
response.json()

[{'txid': '07e555a4400096be80ac7247d26d394ee6a7b733bcd79aab4d90de497d7d4bfc',
  'version': 1,
  'locktime': 0,
  'vin': [{'txid': '6f0899c18c5ae15120c28e80fe28a5d4353e1e077a4d86862e139ae80e5143e1',
    'vout': 0,
    'prevout': {'scriptpubkey': '00200e8758dadd874a46942b5cdf921a3948434ba4933bbe083950a76546ccb606f1',
     'scriptpubkey_asm': 'OP_0 OP_PUSHBYTES_32 0e8758dadd874a46942b5cdf921a3948434ba4933bbe083950a76546ccb606f1',
     'scriptpubkey_type': 'v0_p2wsh',
     'scriptpubkey_address': 'tb1qp6r43kkasa9yd9pttn0eyx3efpp5hfyn8wlqsw2s5aj5dn9kqmcscn29mv',
     'value': 2981655},
    'scriptsig': '',
    'scriptsig_asm': '',
    'witness': ['3045022100d4e6a38d8eb1409d32153a8ba11fd2f8de74e600191c06bc48b352eae998bda30220764dfde3b07ca5d46966f56fcf9f06f9a4086783e3e6834a7ec2dac2eb0fb35101',
     '02da7831ee560737900635f4ffc97f2f0ac452ffa4f9525e6c6368f59938a7fdbd',
     '7765617265616c6c7361746f736869',
     'a91478d574f6f76809749cb351af7e20ae52082e2e378876a91474ad2f4b2ca10e2ca0028d574573ee

In [25]:
'''
We can find the recv address on the mem pool as well (sleep to give transaction a chance to propogate!)
'''
print("https://mempool.space/testnet/address/" + recv['address'])
api_url = "https://mempool.space/testnet/api/address/" + recv['address']
response = requests.get(api_url)
response.json()

https://mempool.space/testnet/address/tb1q90ltvsgaeyxanzz2m9fdeerg9s2mjn709s7ukg


{'address': 'tb1q90ltvsgaeyxanzz2m9fdeerg9s2mjn709s7ukg',
 'chain_stats': {'funded_txo_count': 0,
  'funded_txo_sum': 0,
  'spent_txo_count': 0,
  'spent_txo_sum': 0,
  'tx_count': 0},
 'mempool_stats': {'funded_txo_count': 0,
  'funded_txo_sum': 0,
  'spent_txo_count': 0,
  'spent_txo_sum': 0,
  'tx_count': 0}}

In [26]:
'''
And information on the locking transaction
'''
time.sleep(5)
print("https://mempool.space/testnet/tx/" + locking_tx_id)
api_url = "https://mempool.space/testnet/api/tx/" + locking_tx_id
response = requests.get(api_url)
response.json()

https://mempool.space/testnet/tx/aceb17b4f47885f8c201206a0feb47f8431c9f31d8207e20fc5010c0f67a4b0b


{'txid': 'aceb17b4f47885f8c201206a0feb47f8431c9f31d8207e20fc5010c0f67a4b0b',
 'version': 1,
 'locktime': 0,
 'vin': [{'txid': '07e555a4400096be80ac7247d26d394ee6a7b733bcd79aab4d90de497d7d4bfc',
   'vout': 0,
   'prevout': {'scriptpubkey': '001474ad2f4b2ca10e2ca0028d574573eedd1d065d7f',
    'scriptpubkey_asm': 'OP_0 OP_PUSHBYTES_20 74ad2f4b2ca10e2ca0028d574573eedd1d065d7f',
    'scriptpubkey_type': 'v0_p2wpkh',
    'scriptpubkey_address': 'tb1qwjkj7jev5y8zegqz34t52ulwm5wsvhtleukcfa',
    'value': 2980655},
   'scriptsig': '',
   'scriptsig_asm': '',
   'witness': ['3044022011df3095d857e54fa61f823aea88e767dbf90637a84b949f1b55e3351388fa8f0220259fe9e49a3db7f24accdb6406480154973853f5611e0f9e537876a205fab86401',
    '02da7831ee560737900635f4ffc97f2f0ac452ffa4f9525e6c6368f59938a7fdbd'],
   'is_coinbase': False,
   'sequence': 4294967295}],
 'vout': [{'scriptpubkey': '00200165dc58dd14e441cf3079e135797c4ea1192953b4cd57a9d0314e7f2cb8f81d',
   'scriptpubkey_asm': 'OP_0 OP_PUSHBYTES_32 0165dc58dd1

In [27]:
'''
And redeem transaction
''' 
api_url = "https://mempool.space/testnet/api/tx/" + redeem_tx_id
response = requests.get(api_url)
response.json()

{'txid': '41cd82ced4e980551315aca56501fc02e98568d4b6b7d7ae2de32ece9cea280b',
 'version': 1,
 'locktime': 0,
 'vin': [{'txid': 'aceb17b4f47885f8c201206a0feb47f8431c9f31d8207e20fc5010c0f67a4b0b',
   'vout': 0,
   'prevout': {'scriptpubkey': '00200165dc58dd14e441cf3079e135797c4ea1192953b4cd57a9d0314e7f2cb8f81d',
    'scriptpubkey_asm': 'OP_0 OP_PUSHBYTES_32 0165dc58dd14e441cf3079e135797c4ea1192953b4cd57a9d0314e7f2cb8f81d',
    'scriptpubkey_type': 'v0_p2wsh',
    'scriptpubkey_address': 'tb1qq9jackxaznjyrnes08sn27tuf6s3j22nknx402wsx9887t9clqwsm6tptf',
    'value': 2979655},
   'scriptsig': '',
   'scriptsig_asm': '',
   'witness': ['3045022100eddf6614990330259823b3443ad806d83158869dd6d876e82835eb1f2aad711a022004978dc709549d19dea33a4455ee21b02f1b60f1b6139a83e35eb56aed45a4df01',
    '03bf33f1d559708db5c824d8139ca25f3c55d99b6bd20c844d46596226fdf545d9',
    '7765617265616c6c7361746f736869',
    'a91478d574f6f76809749cb351af7e20ae52082e2e378876a9142bfeb6411dc90dd9884ad952dce4682c15b94fcf88ac']

In [29]:
'''
And we can confirm if it has made it into a block or not
'''
api_transact_url = "https://mempool.space/testnet/api/tx/" + redeem_tx_id + "/status"
response = requests.get(api_transact_url)
response.json()

{'confirmed': False}